# Import dependencies

In [ ]:
%pip install ai-edge-litert qai_hub_models

In [12]:
from qai_hub_models.models.facemap_3dmm.model import MODEL_ASSET_VERSION, MODEL_ID
from qai_hub_models.utils.asset_loaders import CachedWebModelAsset
from ai_edge_litert.interpreter import Interpreter
import os, cv2, numpy as np
from pathlib import Path
import shutil

# Initialize Assets

In [13]:
ASSET_NAMES = [
    "meanFace.npy",
    "shapeBasis.npy",
    "blendShape.npy",
    "face_img.jpg",
    "face_img_fbox.txt",
]

out_dir = Path("./assets")
out_dir.mkdir(parents=True, exist_ok=True)

resolved = {}
for name in ASSET_NAMES:
    src_path = CachedWebModelAsset.from_asset_store(
        MODEL_ID, MODEL_ASSET_VERSION, name
    ).fetch()
    dst_path = out_dir / name
    shutil.copy2(src_path, dst_path)
    resolved[name] = str(dst_path)

print(resolved)

{'meanFace.npy': 'assets/meanFace.npy', 'shapeBasis.npy': 'assets/shapeBasis.npy', 'blendShape.npy': 'assets/blendShape.npy', 'face_img.jpg': 'assets/face_img.jpg', 'face_img_fbox.txt': 'assets/face_img_fbox.txt'}


# Import utility functions

In [19]:
ASSETS_DIR = "./assets"


def _load_assets(assets_dir=ASSETS_DIR):
    face = np.load(Path(assets_dir) / "meanFace.npy").reshape(-1, 1).astype(np.float32)
    basis_id = np.load(Path(assets_dir) / "shapeBasis.npy").astype(np.float32)
    basis_exp = np.load(Path(assets_dir) / "blendShape.npy").astype(np.float32)
    vn = 68
    face = face.reshape(3 * vn, 1)
    basis_id = basis_id.reshape(3 * vn, 219)
    basis_exp = basis_exp.reshape(3 * vn, 39)
    return face, basis_id, basis_exp, vn


def _project(output, face, basis_id, basis_exp, vn):
    a_id = output[0:219]
    a_exp = output[219:258]
    pitch = output[258]
    yaw = output[259]
    roll = output[260]
    tX = output[261]
    tY = output[262]
    f = output[263]
    a_id = a_id * 3.0
    a_exp = a_exp * 0.5 + 0.5
    pitch = pitch * np.pi / 2.0
    yaw = yaw * np.pi / 2.0
    roll = roll * np.pi / 2.0
    tX = tX * 60.0
    tY = tY * 60.0
    tZ = 500.0
    f = f * 150.0 + 450.0
    p = np.array(
        [
            [1.0, 0.0, 0.0],
            [0.0, np.cos(-np.pi), -np.sin(-np.pi)],
            [0.0, np.sin(-np.pi), np.cos(-np.pi)],
        ],
        dtype=np.float32,
    )
    cr, sr = np.cos(-roll), np.sin(-roll)
    cp, sp = np.cos(-pitch), np.sin(-pitch)
    cy, sy = np.cos(-yaw), np.sin(-yaw)
    Rz = np.array([[cr, -sr, 0.0], [sr, cr, 0.0], [0.0, 0.0, 1.0]], dtype=np.float32)
    Ry = np.array([[cy, 0.0, sy], [0.0, 1.0, 0.0], [-sy, 0.0, cy]], dtype=np.float32)
    Rx = np.array([[1.0, 0.0, 0.0], [0.0, cp, -sp], [0.0, sp, cp]], dtype=np.float32)
    R = Ry @ (Rx @ (p @ Rz))
    shape = face + basis_id @ a_id.reshape(-1, 1) + basis_exp @ a_exp.reshape(-1, 1)
    shape = shape.reshape(vn, 3)
    V = shape @ R.T
    V[:, 0] += tX
    V[:, 1] += tY
    V[:, 2] += tZ
    lm = V[:, :2] * (f / tZ)
    return lm.astype(np.float32), float(pitch), float(yaw), float(roll)


def _normalize(inp, mode):
    if mode == "0_1":
        return inp.astype(np.float32) / 255.0
    if mode == "neg1_1":
        return (inp.astype(np.float32) / 127.5) - 1.0
    return inp.astype(np.float32)


def _rect_to_square(x0, y0, x1, y1, H, W, scale=1.1):
    cx = 0.5 * (x0 + x1)
    cy = 0.5 * (y0 + y1)
    side = max(x1 - x0 + 1, y1 - y0 + 1) * scale
    nx0 = int(round(cx - side / 2))
    ny0 = int(round(cy - side / 2))
    nx1 = int(round(cx + side / 2))
    ny1 = int(round(cy + side / 2))
    nx0 = max(0, nx0)
    ny0 = max(0, ny0)
    nx1 = min(W - 1, nx1)
    ny1 = min(H - 1, ny1)
    return nx0, ny0, nx1, ny1


def _prep(
    img,
    bbox,
    ih,
    iw,
    bbox_order="x0y0x1y1",
    square_crop=True,
    square_scale=1.1,
    norm="0_1",
    to_rgb=True,
):
    H, W = img.shape[:2]
    if bbox is None:
        x0, y0, x1, y1 = 0, 0, W - 1, H - 1
    else:
        if bbox_order == "x0x1y0y1":
            x0, x1, y0, y1 = bbox
            x0, y0, x1, y1 = int(x0), int(y0), int(x1), int(y1)
        else:
            x0, y0, x1, y1 = [int(v) for v in bbox]
    if square_crop:
        x0, y0, x1, y1 = _rect_to_square(x0, y0, x1, y1, H, W, square_scale)
    roi = img[y0 : y1 + 1, x0 : x1 + 1]
    if to_rgb:
        roi = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    roi = cv2.resize(roi, (iw, ih), interpolation=cv2.INTER_LINEAR)
    inp = _normalize(roi, norm)[np.newaxis, ...]
    return inp, (x0, y0, x1, y1)


def _transform(lm, bbox, rh, rw):
    x0, y0, x1, y1 = bbox
    h = (y1 - y0) + 1
    w = (x1 - x0) + 1
    out = lm.copy()
    out[:, 0] = (out[:, 0] + rw / 2.0) * (w / rw) + x0
    out[:, 1] = (out[:, 1] + rh / 2.0) * (h / rh) + y0
    return out


def annotate_image(
    image_path,
    interpreter,
    assets_dir=ASSETS_DIR,
    bbox=None,
    save_path=None,
    bbox_order="x0y0x1y1",
    square_crop=True,
    square_scale=1.1,
    norm="0_1",
    to_rgb=True,
    radius=10,
    thickness=-1,
):
    face, basis_id, basis_exp, vn = _load_assets(assets_dir)
    inp_info = interpreter.get_input_details()[0]
    out_info = interpreter.get_output_details()[0]
    ih, iw = int(inp_info["shape"][1]), int(inp_info["shape"][2])
    img = cv2.imread(image_path)
    inp, bbox_xyxy = _prep(
        img, bbox, ih, iw, bbox_order, square_crop, square_scale, norm, to_rgb
    )
    interpreter.set_tensor(inp_info["index"], inp.astype(np.float32))
    interpreter.invoke()
    out = interpreter.get_tensor(out_info["index"])[0]
    lm_crop, pitch, yaw, roll = _project(out, face, basis_id, basis_exp, vn)
    lm_img = _transform(lm_crop, bbox_xyxy, ih, iw)
    vis = img.copy()
    for x, y in lm_img:
        cv2.circle(vis, (int(round(x)), int(round(y))), radius, (0, 255, 0), thickness)
    deg = np.array([pitch, yaw, roll]) * (180.0 / np.pi)
    cv2.putText(
        vis,
        f"pitch:{deg[0]:.1f} yaw:{deg[1]:.1f} roll:{deg[2]:.1f}",
        (bbox_xyxy[0], max(0, bbox_xyxy[1] - 10)),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.55,
        (0, 255, 0),
        1,
        cv2.LINE_AA,
    )
    if save_path:
        Path(save_path).parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(save_path, vis)
    return lm_img, {
        "pitch_rad": pitch,
        "yaw_rad": yaw,
        "roll_rad": roll,
        "pitch_deg": deg[0],
        "yaw_deg": deg[1],
        "roll_deg": deg[2],
    }

# Initialize TFLite Interpreter

In [15]:
MODEL_PATH = "./facemap_3dmm-facial-landmark-detection-float.tflite"
SAMPLE_DIR = "./sample"
RESULTS_DIR = "./results"
CONF_THRESH = 0.5
NUM_LANDMARKS = 68

os.makedirs(RESULTS_DIR, exist_ok=True)

try:
    interpreter = Interpreter(model_path=MODEL_PATH)
    interpreter.allocate_tensors()
    print("✅ Model loaded and tensors allocated successfully.")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    print("Please ensure the model file is at the correct path:", MODEL_PATH)
    exit()

input_details = interpreter.get_input_details()[0]
output_details = interpreter.get_output_details()[0]

INPUT_SHAPE = input_details["shape"]
INPUT_HEIGHT = INPUT_SHAPE[1]
INPUT_WIDTH = INPUT_SHAPE[2]

print("\n--- Model Details ---")
print("Inputs:\n", input_details)
print("Outputs:\n", output_details)
print("---------------------\n")

✅ Model loaded and tensors allocated successfully.

--- Model Details ---
Inputs:
 {'name': 'image', 'index': 0, 'shape': array([  1, 128, 128,   3], dtype=int32), 'shape_signature': array([  1, 128, 128,   3], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
Outputs:
 {'name': 'parameters_3dmm', 'index': 85, 'shape': array([  1, 265], dtype=int32), 'shape_signature': array([  1, 265], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0), 'quantization_parameters': {'scales': array([], dtype=float32), 'zero_points': array([], dtype=int32), 'quantized_dimension': 0}, 'sparsity_parameters': {}}
---------------------



# Fetch image files from sample directory

In [16]:
try:
    print("🔄Loading sample images...")
    image_files = [f for f in os.listdir(SAMPLE_DIR) if f.endswith(".jpg")]
    if not image_files:
        print("⚠️ No .jpg images found in the 'sample' directory.")
except FileNotFoundError:
    print(f"❌ Error: The directory '{SAMPLE_DIR}' was not found.")
    image_files = []
finally:
    print("✅Sample Images Loaded")

🔄Loading sample images...
✅Sample Images Loaded


# Run inference on each sample image (.jpg)

In [20]:
paths = [p for p in Path(SAMPLE_DIR).glob("*.jpg")]
for p in paths:
    out_path = str(Path(RESULTS_DIR) / (p.stem + "_annotated.jpg"))
    lmk, pose = annotate_image(
        str(p), interpreter, assets_dir=ASSETS_DIR, bbox=None, save_path=out_path
    )
    print(
        p.name,
        {
            k: round(v, 2) if isinstance(v, float) else round(float(v), 2)
            for k, v in pose.items()
        },
    )

yaw-left.jpg {'pitch_rad': 0.05, 'yaw_rad': -0.72, 'roll_rad': 0.03, 'pitch_deg': 2.76, 'yaw_deg': -41.2, 'roll_deg': 1.76}
pitch-up.jpg {'pitch_rad': 0.49, 'yaw_rad': -0.03, 'roll_rad': -0.02, 'pitch_deg': 28.28, 'yaw_deg': -1.81, 'roll_deg': -1.32}
roll-left.jpg {'pitch_rad': -0.04, 'yaw_rad': -0.01, 'roll_rad': -0.37, 'pitch_deg': -2.03, 'yaw_deg': -0.42, 'roll_deg': -21.45}
roll-right.jpg {'pitch_rad': -0.11, 'yaw_rad': 0.03, 'roll_rad': 0.46, 'pitch_deg': -6.24, 'yaw_deg': 1.82, 'roll_deg': 26.16}
yaw-right.jpg {'pitch_rad': 0.17, 'yaw_rad': 0.75, 'roll_rad': 0.03, 'pitch_deg': 9.85, 'yaw_deg': 42.98, 'roll_deg': 1.69}
pitch-down.jpg {'pitch_rad': -0.64, 'yaw_rad': 0.02, 'roll_rad': -0.02, 'pitch_deg': -36.88, 'yaw_deg': 1.24, 'roll_deg': -1.32}


In [18]:
import numpy as np

fbox = np.loadtxt("./assets/face_img_fbox.txt")
x0, x1, y0, y1 = [int(v) for v in fbox]
bbox = (x0, y0, x1, y1)
annotate_image(
    "./assets/face_img.jpg",
    interpreter,
    bbox=bbox,
    save_path="./results/qcom_demo_check.jpg",
)

(array([[126.83038, 197.18317],
        [126.00188, 232.7856 ],
        [128.54886, 265.7019 ],
        [134.95789, 303.11304],
        [146.1018 , 327.98413],
        [159.56046, 346.99725],
        [176.70488, 361.6625 ],
        [196.4616 , 374.26166],
        [216.79756, 379.0856 ],
        [237.49966, 377.42395],
        [258.77606, 367.99692],
        [277.66913, 356.1018 ],
        [293.50354, 339.33038],
        [308.1042 , 316.3846 ],
        [319.86957, 280.38083],
        [327.32095, 248.21677],
        [331.89447, 212.90955],
        [159.67911, 188.68365],
        [169.63725, 186.15634],
        [180.50061, 185.62088],
        [192.06578, 187.00038],
        [201.97835, 190.01627],
        [258.63364, 195.50662],
        [268.67697, 194.55695],
        [280.21143, 195.40579],
        [290.87762, 197.74857],
        [300.47906, 201.62383],
        [229.38187, 208.47101],
        [227.98453, 227.60217],
        [226.4234 , 248.89262],
        [225.12665, 265.03357],
        